## __TRABAJO DE INVESTIGACIÓN__ 
***
### __Métodos de Optimización 2022-2__
#### __Profesor__: Cristián Sepulveda S.
#### __Ayudante__: Tomás Lopez A.
#### __Integrantes__:  Estefanía Álvarez M. - Bastián Loyola J. - Carlos Retamales A.

***

A lo largo de este trabajo se busca resolver el problema de optimización del problema del viajante, el cual consiste en encontrar la ruta más corta que recorre todas las ciudades de un conjunto de ciudades, pasando una única vez por cada una de ellas. 
Pero bajo el contexto de puntos de reciclaje, donde se busca encontrar la ruta más corta que recorre todos los puntos de reciclaje de un conjunto de puntos de reciclaje, pasando una única vez por cada uno de ellos, teniendo en cuanta que este recorrido lo hacen camiones de reciclaje con una capacidad máxima de carga.

***

### __Variables__

* $n$: Número de puntos de reciclaje
* $m$: Número de camiones
* $dm$: Número de drones
* $k$: Número de vuelos de cada drone
* $W_i$: Capacidad máxima de carga de cada camión
* $w_i$: Peso del punto de reciclaje $i$ en kilogramos
* $i$: Índice de los puntos de reciclaje
* $j$: Índice de los puntos de reciclaje
* $x_{ij}$: Variable binaria que indica si el camión pasa por el punto de reciclaje $i$ en el recorrido $j$
* $c_{ij}$: Costo de ir del punto de reciclaje $i$ al punto de reciclaje $j$ del camión
* $d_{i}$: Costo de ir del punto de reciclaje $i$ del drone 
* $cd_{ij}$: Costo de ir del punto de reciclaje $i$ al punto de reciclaje $j$ de cada drone


***


### __METODOLOGÍAS__

***

METODOLOGÍA USANDO SOLO CAMIONES

***

### __CASOS DE PRUEBA__

***

In [101]:
#--------------------CASO 1--------------------#
#Se definen los datos del problema
n = 5 #cantidad de puntos limpios
m = 2 #cantidad de camiones
W = [100,120] # capacidad del camión m
w = [0, 10, 10, 30, 50] #peso de cada reciclaje
#costo de transporte de cada residuo desde cada punto limpio
c = [0 0 ;1 1;2 2;3 3;4 4]

5×2 Matrix{Int64}:
 0  0
 1  1
 2  2
 3  3
 4  4

In [ ]:
#--------------------CASO 2--------------------#
#Se definen los datos del problema
n = 5 #cantidad de puntos limpios
m = 4 #cantidad de camiones
W = [100,120,50,200] # capacidad del camión m
w = [0, 100, 10, 30, 50] #peso de cada reciclaje
#costo de transporte de cada residuo desde cada punto limpio
c = [0 0 5 0;1 5 1 1;2 3 2 2;3 4 3 3; 4 4 5 4]

In [102]:
using JuMP, GLPK

#Se define el modelo
model = Model(GLPK.Optimizer)

#Se definen las variables
#binarias
#camión j que recoge el reciclaje del punto limpio i
@variable(model, x[1:n, 1:m], Bin)

#Función objetivo
@objective(model, Min, sum(c[i,j]*x[i,j] for i in 1:n, j in 1:m))

#Restricciones

#1.Un camion puede recoger uno o mas reciclajes
@constraint(model, [j in 1:m], sum(x[i,j] for i in 1:n) >= 1)

#2.Un reciclaje solo puede ser recogido por uno o mas camiones
@constraint(model, [i in 1:n], sum(x[i,j] for j in 1:m) >= 1)

#3.La capacidad del camion no puede ser sobrepasada
@constraint(model, [j in 1:m], sum(w[i]*x[i,j] for i in 1:n) <= W[j])

#4.El camion puede solo pasar una vez por cada punto limpio, excepto el punto de origen
@constraint(model, [i in 2:n, j in 1:m], x[i,j] <= sum(x[k,j] for k in 1:n if k != i))

#5.El camion puede pasar por el punto de origen cuantas veces sea necesario
@constraint(model, [j in 1:m], sum(x[1,j] for i in 1:n) >= 1)

#6.El camion si pasa por el punto de origen su capacidad se restablece a W
@constraint(model, [j in 1:m], sum(w[i]*x[i,j] for i in 1:n) <= W[j] + W[j]*x[1,j])

#7. El camion puede pasar de un punto limpio al punto de origen
@constraint(model, [j in 1:m], x[1,j] <= sum(x[i,j] for i in 1:n))

#8. El camion tiene que pasar por el punto de origen al finalizar su recorrido
@constraint(model, [j in 1:m], sum(x[i,j] for i in 1:n) >= 1)

#Se resuelve el modelo
optimize!(model)
#Se imprime el resultado
println("Valor óptimo: ", objective_value(model))

#Se imprime la ruta de cada camión
println("Ruta de cada camión")
for j in 1:m
    println("El camión ", j, " sigue la ruta:")
    #Formato
        #Punto 1 -> Punto 2 -> ... -> Punto n
    print("\t")
    for i in 1:n
        if value(x[i,j]) == 1
            print("Punto ", i, " -> ")
        end
    end
    println("Punto 1")
    
end


Valor óptimo: 10.0
Ruta de cada camión
El camión 1 sigue la ruta:
	Punto 1 -> Punto 3 -> Punto 4 -> Punto 5 -> Punto 1
El camión 2 sigue la ruta:
	Punto 1 -> Punto 2 -> Punto 1


***

METODOLOGÍA USANDO SOLO CAMIONES Y UN DRON

***

### __CASOS DE PRUEBA__

***

In [99]:
#--------------------CASO 1--------------------#
# Número de camiones, puntos de reciclaje y vuelos del dron
m = 3
n = 5
k = 2

# Capacidad de los camiones y peso de los puntos de reciclaje
W = [50, 80, 150]
w = [20, 30, 10, 15, 25]

# Matrices de costos de transporte
c = [10 20 30 40 50;
     15 25 35 45 55;
     20 15 40 50 60]
d = [2, 3, 4, 5, 6]

5-element Vector{Int64}:
 2
 3
 4
 5
 6

In [100]:
using JuMP, GLPK


# Crear el modelo
model = Model(with_optimizer(GLPK.Optimizer))

# Definir las variables
@variable(model, x[1:m, 1:n], Bin) # variables binarias que indican si el camión i visita el punto j
@variable(model, y[1:n], Bin) # variables binarias que indican si el dron visita el punto j
@variable(model, z[1:m], Bin) # variables binarias que indican si el camión i vuelve al centro de reciclaje
@variable(model, u[1:m], Bin) # variables binarias que indican si el camión i o el dron regresan al centro de reciclaje

# Definir la función objetivo
@objective(model, Min, sum(c[i,j]*x[i,j] for i in 1:m, j in 1:n) + sum(d[j]*y[j] for j in 1:n)) # minimizar el costo total de transporte

# Definir las restricciones
#1. Cada punto de reciclaje debe ser visitado por exactamente un medio de transporte (camión o dron)
@constraint(model, [j in 1:n], sum(x[i,j] for i in 1:m) + y[j] == 1)

#2. Los camiones deben respetar su capacidad de carga
@constraint(model, [i in 1:m], sum(w[j]*x[i,j] for j in 1:n) - z[i]*W[i] <= W[i])

#3. El dron solo puede realizar k vuelos
@constraint(model, sum(y[j] for j in 1:n) <= k)

#4. Los camiones deben regresar al centro de reciclaje para restablecer su capacidad
@constraint(model, [i in 1:m], z[i]*W[i] <= sum(w[j]*x[i,j] for j in 1:n))

#5. Todos los medios de transporte deben regresar al centro de reciclaje
@constraint(model, sum(u[i] for i in 1:m) + sum(y[j] for j in 1:n) == 1)

#Resolver el modelo
optimize!(model)

# Imprimir el resultado
println("Costo total: ", objective_value(model))

# Recorrer cada camión
for i in 1:m
    println("Camión ", i, ":")
    ruta = [] # lista para guardar la ruta del camión i
    
    # Recorrer cada punto de reciclaje
    for j in 1:n
        if value(x[i,j]) == 1 # si el camión i visita el punto j
            println("- Punto de reciclaje ", j)
            push!(ruta, j) # añadir el punto j a la ruta del camión i
        end
    end
    
    # Si el camión i vuelve al centro de reciclaje, añadirlo a la ruta
    if value(z[i]) == 1
        println("- Centro de reciclaje (origen)")
        push!(ruta, 0) # añadir el centro de reciclaje a la ruta del camión i
    end
    # Imprimir la ruta completa del camión i
    println("Ruta: ", ruta)
end

# Si el dron regresa al centro de reciclaje, imprimirlo
if value(u[1]) == 1
    println("Dron: Centro de reciclaje (origen)")

# Si el dron visita algún punto de reciclaje, imprimirlo
else
    for j in 1:n
        if value(y[j]) == 1
            println("Dron: Punto de reciclaje ", j)
        end
    end
end




Costo total: 100.99999999999999
Camión 1:
- Punto de reciclaje 1
- Punto de reciclaje 3
- Punto de reciclaje 4
Ruta: Any[1, 3, 4]
Camión 2:
Ruta: Any[]
Camión 3:
- Punto de reciclaje 2
Ruta: Any[2]
Dron: Punto de reciclaje 5


***

METODOLOGÍA USANDO SOLO CAMIONES Y DRONES

***

### __CASOS DE PRUEBA__

***

In [96]:
#--------------------CASO 1--------------------#
#Se definen los datos del problema
n = 5 #cantidad de puntos limpios
m = 4 #cantidad de camiones
dm = 3 #cantidad de drones
k = [2, 3, 5] #cantidad de vuelos que pueden realizar los drones
W = [100,120,50,200] # capacidad del camión m
w = [0, 100, 10, 30, 50] #peso de cada reciclaje
#costo de transporte de cada residuo desde cada punto limpio para los camiones
c = [0 0 5 0;1 5 1 1;2 3 2 2;3 4 3 3; 4 4 5 4]
#costo de transporte de cada residuo desde cada punto limpio para los drones
cd = [0 0 3 0;1 3 1 1;1 1 1 1;3 3 3 3; 4 3 5 3]

5×4 Matrix{Int64}:
 0  0  3  0
 1  3  1  1
 1  1  1  1
 3  3  3  3
 4  3  5  3

In [98]:
using JuMP, GLPK


#Se define el modelo
model = Model(GLPK.Optimizer)

#Se definen las variables
#binarias
#camión j que recoge el reciclaje del punto limpio i
@variable(model, x[1:n, 1:m], Bin)
#drone d que recoge el reciclaje del punto limpio i
@variable(model, yd[1:n, 1:dm], Bin)

#Función objetivo
@objective(model, Min, sum(c[i,j]*x[i,j] for i in 1:n, j in 1:m) + sum(cd[i,d]*yd[i,d] for i in 1:n, d in 1:dm))

#Restricciones

#1.Un camion puede recoger uno o mas reciclajes
@constraint(model, [j in 1:m], sum(x[i,j] for i in 1:n) >= 1)

#2.Un reciclaje solo puede ser recogido por uno o mas camiones o drones
@constraint(model, [i in 1:n], sum(x[i,j] for j in 1:m) + sum(yd[i,d] for d in 1:dm) >= 1)

#3.La capacidad del camion no puede ser sobrepasada
@constraint(model, [j in 1:m], sum(w[i]*x[i,j] for i in 1:n) <= W[j])

#4.El camion puede solo pasar una vez por cada punto limpio, excepto el punto de origen
@constraint(model, [i in 2:n, j in 1:m], x[i,j] <= sum(x[k,j] for k in 1:n if k != i))

#5.El camion puede pasar por el punto de origen cuantas veces sea necesario
@constraint(model, [j in 1:m], sum(x[1,j] for i in 1:n) >= 1)

#6.El camion si pasa por el punto de origen su capacidad se restablece a W
@constraint(model, [j in 1:m], sum(w[i]*x[i,j] for i in 1:n) <= W[j] + W[j]*x[1,j])

#7. El camion puede pasar de un punto limpio al punto de origen
@constraint(model, [j in 1:m], x[1,j] <= sum(x[i,j] for i in 1:n))

#8. El camion tiene que pasar por el punto de origen al finalizar su recorrido
@constraint(model, [j in 1:m], sum(x[i,j] for i in 1:n) >= 1)

#9. Cada drone solo puede visitar una cantidad limitada de puntos de reciclaje
@constraint(model, [d in 1:dm], sum(yd[i,d] for i in 1:n) <= k[d])

#10. Cada drone solo puede visitar puntos de reciclaje que no hayan sido visitados por otros drones
@constraint(model, [d in 1:dm, i in 2:n], yd[i,d] <= sum(1 - yd[j,d] for j in 1:(i-1)))

#11. Cada punto de reciclaje solo puede ser visitado por un drone
@constraint(model, [i in 1:n], sum(yd[i,d] for d in 1:dm) <= 1)

#Se resuelve el modelo
optimize!(model)

#Se imprime la solución
println("La función objetivo es: ", objective_value(model))

#Se imprime la ruta de cada camion
for j in 1:m
    println("Ruta del camión ", j, ":")
    for i in 1:n
        if value(x[i,j]) == 1
            println("\tPunto de reciclaje ", i)
        end
    end
end

#Se imprime la ruta de cada drone
for d in 1:dm
    println("Ruta del drone ", d, ":")
    for i in 1:n
        if value(yd[i,d]) == 1
            println("\tPunto de reciclaje ", i)
        end
    end
end


La función objetivo es: 12.999999999999996
Ruta del camión 1:
	Punto de reciclaje 1
	Punto de reciclaje 2
Ruta del camión 2:
	Punto de reciclaje 1
Ruta del camión 3:
	Punto de reciclaje 1
Ruta del camión 4:
	Punto de reciclaje 1
	Punto de reciclaje 4
Ruta del drone 1:
	Punto de reciclaje 3
Ruta del drone 2:
	Punto de reciclaje 5
Ruta del drone 3:


***